In [41]:
import cv2 
import re
import pandas as pd
import numpy as np
import shutil
import os
import glob
from datetime import date
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

In [42]:
#book1 = pd.read_csv(r'C:\Users\shardul\Hindustan Times\Result.csv')
#book2 = pd.read_excel(r'D:\job\MIB Print  Keywords.xlsx')

In [43]:
def imgProcess(imgPath):
    black = (0,0,0)
    white = (255,255,255)
    threshold = (160,160,160)

    # Open input image in grayscale mode and get its pixels.
    img = Image.open(imgPath).convert("LA")
    pixels = img.getdata()

    newPixels = []

    # Compare each pixel 
    for pixel in pixels:
        if pixel < threshold:
            newPixels.append(black)
        else:
            newPixels.append(white)

    # Create and save new image.
    newImg = Image.new("RGB",img.size)
    newImg.putdata(newPixels)
    #newImg.save("samplechk.jpg")
    return newImg

In [44]:
def gen_Author(image_path, language = "eng"):
    source=""
    #print("hello")
    #Author extraction
    img_cv = cv2.imread(image_path)
    #img_cv = imgProcess(image_path)
    try:
        #img_cv = get_grayscale(img_cv)
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    except:
        pass
    custom_config = r'--oem 3'
    text = pytesseract.image_to_string(img_cv, lang = language, config=custom_config)
    outputList = text.split('\n')
    while outputList.count('') != 0:
        outputList.remove('')
    for i in range(len(outputList)):
        if outputList[i].count('@') > 0:
            outputList[i] = outputList[i].replace(' ','')
    #print(outputList)
    flag = False
    for i in range(len(outputList)):
        line = outputList[i]
        match = re.findall(r'[\w\.-]+@[\w\.-]+', line)
        if len(match) != 0:
            source='Journalist'
            #print(source)
            flag = True
            author = outputList[i-1]
            mailAuthor = outputList[i]
            break
    if not flag:
        source='Agency'
        #print(source)
        author = "Bureau"
        mailAuthor = "Bureau"
    return author, mailAuthor,source

In [45]:
def gen_Headline(image_path, n = 4, language = "eng"):
    img_cv = cv2.imread(image_path)
    image = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    kernel = np.ones((n,n), np.uint8)  # for english 4,4 for hindi 6,6
    temp_image = cv2.dilate(image, kernel = kernel)
    
    boxes = pytesseract.image_to_data(temp_image, lang = language)
    flag1 = False
    #print(boxes)
    coordinates = list()
    for x, b in enumerate(boxes.splitlines()):
        if x != 0:
            b = b.split()
            if len(b) == 12 and b[2] == '1':
                x, y, w, h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
                cv2.rectangle(image, (x-6,y-6), (w+x+6,h+y+6), (0,0,255), 2)
                coordinates.append((x,y,w,h))            
    image_copy = image.copy()
    headline = ""
    try:
        for c in coordinates:
            x, y, w, h = c
            image_roi = image_copy[y-5:y + h+4, x-4:x + w+4] 
            text = pytesseract.image_to_string(image_roi,lang = language, config =  r'--oem 3 --psm 7') # config important otherwise accuracy low
            headline += text

        headline = headline.replace('\n\x0c',' ').strip()
    except:
        #print("yo nub")
        pass
    
    return headline

In [46]:
def thinHeadline(image_path):
    img_cv = cv2.imread(image_path)
    try:
        #img_cv = get_grayscale(img_cv)
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    except:
        pass
    
    custom_config = r'--oem 3'
    text = pytesseract.image_to_string(img_cv, config=custom_config)
    
    outputList = text.split('\n\n')
    while outputList.count('') != 0:
        outputList.remove('')
    while outputList.count(' ') != 0:
        outputList.remove(' ')
    #print(outputList)
    headline = outputList[1]
    headline = headline.replace('\n',' ')
    
    return headline
    

In [47]:
def getHeadlineutil(image_path):
    x = ''
    n = 4
    x = gen_Headline(image_path, n = n)
    author, mailAuthor, source = gen_Author(image_path)
    if author.count(' ') > 1:
            author = mailAuthor.split('@')[0]            
    if x == '':
        #print("anck")
        x = thinHeadline(image_path)
    try:x = x.replace('\x0c','')
    except:pass
    return x , author, source


In [48]:
def genMinistryutil(image_path,book2):
    img_cv = cv2.imread(image_path)
    custom_config = r'--oem 3'
    text = pytesseract.image_to_string(img_cv, lang='eng', config=custom_config)
    flag1= False
    for keyword in book2.Keywords:
        keyword = str(keyword)
        if text.count(keyword) > 0:
            flag1= True
            src= image_path
            dst= "D:/job/data/PMO.foo"
            shutil.copy(src, dst)
            break
    return flag1

In [49]:
def genMinistrymain():
    region=[]
    page_number=[]
    count=0
    ministry='Prime Ministers Office'
    book1=pd.read_excel(r'D:\job\data\ZONE_ZONE.xlsx')
    book2 = pd.read_excel(r'D:\job\keywords\Prime Minister Office.xlsx')
    df = pd.DataFrame(columns = ['Region','page_number','path','headline', 
                                 'author','date','source','Zone','Ministry','Language','article_id']) 
    entries = os.listdir('D:/job/data/newsimages/')
    for entry in entries:
        count+=1
        region=entry
        try:
            zone = book1[book1.Region == region]['Zone']
            zone = list(zone)[0]
            print(zone)
        except:
            zone = 'India'
        pages = os.listdir('D:/job/data/newsimages/'+entry)
        for page in pages:
            #print(page)
            page_number=page
            #print(page_number)
            image= os.listdir('D:/job/data/newsimages/'+entry+'/'+page)
            for image_path in image:
                print(image_path)
                flag_headline = genMinistryutil('D:/job/data/newsimages/'+entry+'/'+page+'/'+image_path,book2)
                print(str("flag_headline")+str(flag_headline))
                if flag_headline:
                    headline, author,source = getHeadlineutil('D:/job/data/newsimages/'+entry+'/'+page+'/'+image_path)
                    x = image_path.split("_", 2)
                    x = list(x)[0]
                    df = df.append({'path' : image_path, 'Region' : region, 'page_number':page,'headline' : 
                                    headline, 'author' : author, 'date':str(date.today()),'source':source,'Zone':zone,
                                    'Ministry':ministry,'Language':'English','article_id':x}, ignore_index = True)
    df.to_csv('PMO.csv')
            

In [50]:
genMinistrymain()

South
570774_179294_10 (1).jpg
flag_headlineFalse
a27b02_179302_9.jpg
flag_headlineFalse
a881bb_180877_5.jpg
flag_headlineFalse
a9004e_179296_1.jpg
flag_headlineFalse
a9ed09_180890_3.jpg
flag_headlineFalse
aacab5_179295_11_mr.jpg
flag_headlineFalse
ab6973_179298_10.jpg
flag_headlineFalse
ac8bd3_180891_1.jpg
flag_headlineFalse
afac49_180887_1.jpg
flag_headlineFalse
b0a092_179295_10.jpg
flag_headlineFalse
b24fa0_179297_2.jpg
flag_headlineFalse
b8b6af_180890_2.jpg
flag_headlineFalse
c64b3c2f-7dfe-4671-a240-f9b3e9e31f7a.jpg
flag_headlineFalse
cb5f79_179294_22.jpg
flag_headlineFalse
cee920_180884_22_mr.jpg
flag_headlineFalse
e93d7e_179302_1.jpg
flag_headlineFalse
ee3cbe_179298_8.jpg
flag_headlineFalse
f90cba_180893_4_mr.jpg
flag_headlineFalse
clip2.jpg
flag_headlineFalse
d4444d_180877_1.jpg
flag_headlineFalse
d74996_179294_19.jpg
flag_headlineFalse
da85c3_179302_11 (1).jpg
flag_headlineTrue
da85c3_179302_11.jpg
flag_headlineTrue
db9d50_179297_9.jpg
flag_headlineFalse
dc9605_179299_5.jpg
fla